In [ ]:
import os, logging, pandas

import tensorflow as tf

logging.getLogger("tensorflow").setLevel(logging.ERROR)

In [ ]:
if not os.path.isdir("/tmp/data"):

  os.makedirs("/tmp/data")

In [ ]:
!gsutil cp gs://cloud-training-demos/feat_eng/data/taxi*.csv /tmp/data

In [ ]:
os.listdir("/tmp/data")

In [ ]:
sample = pandas.read_table('/tmp/data/taxi-train.csv', sep=',')

sample.shape[1]

In [ ]:
sample.columns.tolist()[:4]

In [ ]:
sample.columns.tolist()[4:]

In [ ]:
LabelColumn = sample.columns.tolist()[0]

FeatureCols = sample.columns.tolist()[1:]

len(FeatureCols)

In [ ]:
from tensorflow.data.experimental import make_csv_dataset

def FeatureLabel(dataset):
  """
  Feature & Label
  """
  label = dataset.pop(LabelColumn)

  return dataset, label

def CreateShuffleDataset(pattern, batches, mode='eval'):
  """
  Create Shuffle Dataset

  >>> CreateShuffleDataset('taxi-train.csv', 2, 'train')

  >>> CreateShuffleDataset('taxi-train.csv', 2)
  """
  dataset = make_csv_dataset(pattern, batches)
  dataset = dataset.map(FeatureLabel).cache()
  if mode == 'train':
    # Shuffle Train Dataset
    dataset = dataset.shuffle(1000).repeat()
    dataset = dataset.prefetch(1)
  return dataset

In [ ]:
from tensorflow.keras import layers, models

from tensorflow import feature_column as fc

def BuildModel():
  """
  Functional Model
  """
  inputs = {}; feature_columns = {}
  # Input Layer
  for colname in FeatureCols:
    inputs[colname] = layers.Input(name=colname, shape=(), dtype='float32')
  # Set Feature Columns
  for colname in FeatureCols:
    feature_columns[colname] = fc.numeric_column(colname)
  # Hidden Layer
  # Dense Features Error to Deprecate, Find Something Similar in Function !
  dnn_inputs = layers.DenseFeatures(feature_columns.values())(inputs)
  first_layer = layers.Dense(32, activation='relu')(dnn_inputs)
  second_layer = layers.Dense(8, activation='relu')(first_layer)
  output = layers.Dense(1, activation='linear')(second_layer)
  # Create Model
  model = models.Model(inputs, output)
  # Model Compile
  model.compile(optimizer='adam', loss='mse', metrics=['mse'])
  return model

In [ ]:
TrainExamples = len(pandas.read_table('/tmp/data/taxi-train.csv', sep=','))

ValidExamples = len(pandas.read_table('/tmp/data/taxi-valid.csv', sep=','))

TrainBatchSize = 32